In [1]:
import mne
import os
import numpy as np
import pandas as pd

from mne.time_frequency import tfr_multitaper, tfr_morlet, psd_array_welch
#from mne_features.univariate import compute_spect_entropy
from scipy.stats import kurtosis, skew

Read file


In [2]:
path_file = '/Users/sobieddch/UdG-TFM/ds004504/derivatives/sub-002/eeg/sub-002_task-eyesclosed_eeg.set'
raw = mne.io.read_raw_eeglab(path_file, preload=False, verbose='CRITICAL')
raw

<RawEEGLAB | sub-002_task-eyesclosed_eeg.set, 19 x 396550 (793.1 s), ~57.5 MB, data loaded>

Epochs


In [3]:
sfreq = raw.info['sfreq']

start_time = 0
end_time = 600
duration = 60.0
overlapping = 30.0

epochs = mne.make_fixed_length_epochs(raw.copy().crop(tmin=start_time, tmax=end_time), duration=duration, overlap=overlapping, preload=True, verbose='CRITICAL')
epochs

Number of events,19
Events,1: 19
Time range,0.000 – 59.998 s
Baseline,off


In [9]:
frequency_bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 45)
}

def get_Features_PSD(raw, channels=None, start_time = 0, end_time = 600, duration = 60.0, overlapping = 30.0):
    # variables
    sfreq = raw.info['sfreq']
    
    # Select channels
    if channels==None: ch = raw.ch_names
    else: ch = channels        
    
    # Get Epochs Data
    data = mne.make_fixed_length_epochs(raw.copy().pick(ch).crop(tmin=start_time, tmax=end_time),
                                    duration=duration, overlap=overlapping, preload=True, verbose='CRITICAL').get_data()
    # Compute features
    results = {}
    for band, (fmin,fmax) in frequency_bands.items():
        # Apply bandpass filter to select only the frequency band range
        data_filtered = mne.filter.filter_data(data, sfreq=sfreq, l_freq=fmin, h_freq=fmax, verbose='CRITICAL')
        # Compute the PSD using Welch's method
        psd_all, freqs_all = psd_array_welch(data, sfreq=sfreq, verbose='CRITICAL')
        psd, freqs = psd_array_welch(data_filtered, sfreq=sfreq, verbose='CRITICAL')
        
        # Compute metrics
        total_power = psd.sum()
        relative_power = total_power / psd_all.sum()
        average_power = psd.mean()
        #entropy = compute_spect_entropy(data_filtered, sfreq=sfreq)
        entropy = -np.sum(np.log(psd) * psd)
        std_dev = np.std(data_filtered)
        peak_to_peak = np.ptp(data_filtered)
        kurt = kurtosis(data_filtered)
        skewness = skew(data_filtered)
        # Store results
        results[band] = {
            'total_power': total_power,
            'relative_power': relative_power,
            'average_power': average_power,
            'spectral_entropy': entropy,
            'std': std_dev,
            'peak_to_peak': peak_to_peak,
            'kurtosis': np.mean(kurt),
            'skewness': np.mean(skewness)
        }
    
    #metrics = [f'{main_key}_{nested_key}' for main_key, nested_dict in results_fp1.items() for nested_key in nested_dict.keys()]
    #values = [value for main_key, nested_dict in results_fp1.items() for value in nested_dict.values()]
    
    return results

In [10]:
results_fp1 = get_Features_PSD(raw, channels=['Fp1'])
results_all = get_Features_PSD(raw)

Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 3301 samples (6.602 s)



Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 825 samples (1.650 s)

Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.00 Hz
- Upper transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 14.62 Hz)
- Filter 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 3301 samples (6.602 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 361 out of 361 | elapsed:    0.2s finished


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 361 out of 361 | elapsed:    0.2s finished


Setting up band-pass filter from 13 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 13.00
- Lower transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 11.38 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 509 samples (1.018 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 361 out of 361 | elapsed:    0.1s finished


Setting up band-pass filter from 30 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 221 samples (0.442 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 361 out of 361 | elapsed:    0.2s finished


In [11]:
path_file = '/Users/sobieddch/UdG-TFM/ds004504/derivatives/sub-067/eeg/sub-067_task-eyesclosed_eeg.set'
raw2 = mne.io.read_raw_eeglab(path_file, preload=False, verbose='CRITICAL')
raw2

<RawEEGLAB | sub-067_task-eyesclosed_eeg.set, 19 x 321550 (643.1 s), ~46.6 MB, data loaded>

In [12]:
results2_fp1 = get_Features_PSD(raw2, channels=['Fp1'])
results2_all = get_Features_PSD(raw2)

Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 3301 samples (6.602 s)



Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 825 samples (1.650 s)

Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.00 Hz
- Upper transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 14.62 Hz)
- Filter 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 221 samples (0.442 s)

Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 3301 samples (6.602 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 361 out of 361 | elapsed:    0.2s finished


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 361 out of 361 | elapsed:    0.2s finished


Setting up band-pass filter from 13 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 13.00
- Lower transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 11.38 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 509 samples (1.018 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 361 out of 361 | elapsed:    0.1s finished


Setting up band-pass filter from 30 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 221 samples (0.442 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 361 out of 361 | elapsed:    0.2s finished


Comparison


In [13]:
print(results_fp1['alpha'])
print(results2_fp1['alpha'])

{'total_power': 3.336569476807225e-10, 'relative_power': 0.1075597516961631, 'average_power': 1.3613094560616993e-13, 'spectral_entropy': 8.70257954556739e-09, 'std': 5.836209307805536e-06, 'peak_to_peak': 5.908582730364674e-05, 'kurtosis': -0.1251054616120046, 'skewness': -5.2902386340757464e-05}
{'total_power': 4.691713946889086e-10, 'relative_power': 0.08507585632630149, 'average_power': 1.9142039766989334e-13, 'spectral_entropy': 1.219010026955677e-08, 'std': 6.941888370332027e-06, 'peak_to_peak': 0.00011451900992373648, 'kurtosis': 0.07728312663147047, 'skewness': -0.000212425648464556}


---


## TFR


In [15]:
tfr = tfr_multitaper(epochs.copy(), freqs=np.arange(frequency_bands['theta'][0], frequency_bands['theta'][1], 1), n_cycles=7, return_itc=False, verbose='critical')

In [16]:
tfr.data

array([[[5.20698949e-09, 5.24066393e-09, 5.27411406e-09, ...,
         6.03026768e-09, 6.01427671e-09, 5.99884629e-09],
        [3.52050504e-09, 3.52319860e-09, 3.52562392e-09, ...,
         4.42067768e-09, 4.41254591e-09, 4.40435942e-09],
        [2.59382490e-09, 2.61262182e-09, 2.63176692e-09, ...,
         2.98694801e-09, 2.97899320e-09, 2.97193919e-09],
        [2.67006930e-09, 2.68037920e-09, 2.68963295e-09, ...,
         3.57567104e-09, 3.56461686e-09, 3.55326384e-09]],

       [[5.24955276e-09, 5.28237725e-09, 5.31444330e-09, ...,
         5.75298698e-09, 5.74288567e-09, 5.73259496e-09],
        [3.27255802e-09, 3.28006274e-09, 3.28705244e-09, ...,
         4.01483521e-09, 4.01295600e-09, 4.01029004e-09],
        [2.65460762e-09, 2.67982437e-09, 2.70521113e-09, ...,
         2.96276693e-09, 2.95819807e-09, 2.95349325e-09],
        [2.84607309e-09, 2.85661191e-09, 2.86663485e-09, ...,
         3.79024311e-09, 3.77729688e-09, 3.76373861e-09]],

       [[5.29957445e-09, 5.33247728e

In [17]:
tfr.data.sum()

0.014982802228134324

In [18]:
#np.mean(tfr.data)
tfr.data.mean()

6.571404486023826e-09

In [19]:
kurtosis(tfr.data, axis=(1,2))

array([ 0.23293396,  0.06126115,  0.06579434,  0.10804024, -0.11344009,
       -0.2540888 ,  0.48077593,  0.51818699, -0.73576247, -0.59753481,
        0.29575798,  0.03374003,  0.19627011, -0.10029733, -0.40640983,
       -0.52679684,  0.08685125, -0.19793916,  0.53189139])

In [22]:
def get_Features_TFR(raw, channels=None, start_time = 0, end_time = 600, duration = 60.0, overlapping = 30.0, n_cycles=7, time_bandwidth=4.0):
    # variables
    sfreq = raw.info['sfreq']
    
    # Select channels
    if channels==None: ch = raw.ch_names
    else: ch = channels        
    
    # Get Epochs Data
    epochs = mne.make_fixed_length_epochs(raw.copy().pick(ch).crop(tmin=start_time, tmax=end_time),
                                    duration=duration, overlap=overlapping, preload=True, verbose='CRITICAL')
    
    # Compute features
    results = {}
    for band, (fmin,fmax) in frequency_bands.items():
        # Get the TFR object
        tfr = tfr_multitaper(epochs.copy().pick(ch), freqs=np.arange(fmin,fmax,1), n_cycles=n_cycles, time_bandwidth=time_bandwidth, return_itc=False, verbose='critical')
        # Compute features
        total_power = tfr.data.sum()
        average_power = tfr.data.mean()
        peak_power = tfr.data.max()
        std_dev = tfr.data.std()
        skewness = skew(tfr.data, axis=(1,2))
        kurt = kurtosis(tfr.data, axis=(1,2))
        # Store results
        results[f'{band}_total_power'] = total_power
        results[f'{band}_average_power'] = average_power
        results[f'{band}_peak_power'] = peak_power
        results[f'{band}_std'] = std_dev
        results[f'{band}_kurtosis'] = np.mean(kurt)
        results[f'{band}_skewness'] = np.mean(skewness)
    
    return results

In [23]:
tfr_results = get_Features_TFR(raw, channels=['Fp1'])
tfr_results

{'delta_total_power': 0.019381931071842277,
 'delta_average_power': 1.615160922653523e-07,
 'delta_peak_power': 6.714592259707862e-07,
 'delta_std': 2.17037647797898e-07,
 'delta_kurtosis': -0.32108539356355337,
 'delta_skewness': 1.2244486095742626,
 'theta_total_power': 0.0007447824105880776,
 'theta_average_power': 6.20652008823398e-09,
 'theta_peak_power': 1.3716200970207192e-08,
 'theta_std': 2.2186729367690723e-09,
 'theta_kurtosis': 0.23293396320498072,
 'theta_skewness': 1.0537821447068403,
 'alpha_total_power': 0.00044341224423544054,
 'alpha_average_power': 2.9560816282362702e-09,
 'alpha_peak_power': 6.748293192793814e-09,
 'alpha_std': 1.280930549032744e-09,
 'alpha_kurtosis': -0.8156831444484633,
 'alpha_skewness': 0.19861260693369817,
 'beta_total_power': 0.00024643569885988926,
 'beta_average_power': 4.832072526664496e-10,
 'beta_peak_power': 3.6124994686155768e-09,
 'beta_std': 2.6191528461941027e-10,
 'beta_kurtosis': 18.53449855618216,
 'beta_skewness': 2.747591013664